In [34]:
open System.Collections.Generic
open System.IO

let input = File.ReadAllLines("./input.txt")

In [35]:
type ISizeable = 
    abstract Key: string
    abstract Size: int 

type Command = 
  | ChangeDirectory of path: string * name: string
  | ListDirectory of path: string
  | DirectoryDescriptor of path: string * name: string
  | FileDescriptor of path: string * size: int * name: string

  interface ISizeable with 
    member this.Key = 
        match this with 
        | ChangeDirectory(p,n) -> p 
        | ListDirectory p -> p 
        | DirectoryDescriptor(p,n) -> p
        | FileDescriptor(p,s,n) -> p 

    member this.Size = 
        match this with 
        | FileDescriptor(p,s,n) -> s 
        | _ -> 0    

In [36]:
let getAbsolutePath (pwd: string) (relativePath: string) = 
    let slugs = pwd.Split("/")
    match pwd, relativePath with
    | (pwd, relativePath) when relativePath = "/" -> "/"
    | (pwd, relativePath) when pwd = "/" -> $"/{relativePath}"
    | (pwd, relativePath) when relativePath = ".." -> 
        if slugs.Length = 1 
        then "/" 
        else String.Join("/", slugs[0..slugs.Length-2])
    | (pwd, relativePath) -> $"{pwd}/{relativePath}"


let parseCommand (pwd: byref<_>) (i: string) : ISizeable = 
    let words = i.Split(" ")
    match i, words with 
    | (i, words) when i.StartsWith("$ cd") -> 
        pwd <- getAbsolutePath pwd words[2]
        ChangeDirectory (path = pwd, name = words[2])
    | (i, words) when i.StartsWith("$ ls") -> ListDirectory pwd
    | (i, words) when i.StartsWith("dir") ->  DirectoryDescriptor (path = pwd, name = words[1])
    | (i, words) -> FileDescriptor (path = pwd, size = (words.[0] |> int), name = words[1])

In [70]:
// part one
let groupedInput = 
    let mutable pwd = "/"
    input 
    |> Array.map (fun i -> parseCommand &pwd i)  
    |> Array.groupBy (fun (i:ISizeable) -> i.Key)
    |> Array.map (fun (key:string, commands:ISizeable array) -> (key, Array.sum (Array.map (fun (c:ISizeable) -> c.Size) commands)))

let getIndirect (path:string) = Array.filter (fun (k:string,_) -> k.Contains(path)) groupedInput

let directorySizes = 
    groupedInput
    |> Array.map (fun (key:string, size: int) -> (key, getIndirect key))
    |> Array.map (fun (key:string, sizes:(string * int) array) -> (key, Array.sum (Array.map (fun (_,s) -> s) sizes)))
    
directorySizes
    |> Array.filter (fun (_,s) -> s <= 100000)
    |> Array.sumBy (fun (_,s) -> s)


1325919

In [88]:
// part two 
let sortedDirs = Array.sortBy (fun (_,s) -> s) directorySizes
let spaceUsed = 70_000_000 - snd (Array.last sortedDirs)
let spaceNeeded = 30_000_000 - spaceUsed

sortedDirs
    |> Array.filter (fun (_,s) -> s >= spaceNeeded)
    |> Array.head 
    |> snd 

2050735